In [ ]:
import requests
import pandas as pd

def get_global_earthquakes(min_magnitude=3.5, limit=10000, save_path="global_earthquakes.csv"):
   
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    params = {
        "format": "geojson",
        "minmagnitude": min_magnitude,
        "limit": limit,
        "orderby": "time"
    }
    
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print("⚠️ USGS API 요청 실패! 상태 코드:", response.status_code)
        return pd.DataFrame()  

    data = response.json()

    earthquakes = []
    for feature in data.get("features", []):
        properties = feature["properties"]
        geometry = feature["geometry"]
        earthquakes.append({
            "time": properties["time"],
            "magnitude": properties["mag"],
            "place": properties["place"],
            "lat": geometry["coordinates"][1],
            "lon": geometry["coordinates"][0],
            "depth": geometry["coordinates"][2]
        })

    df = pd.DataFrame(earthquakes)

    # CSV 파일로 저장
    df.to_csv(save_path, index=False, encoding="utf-8")
    print(f" 데이터가 '{save_path}' 파일로 저장되었습니다!")

    return df


df_global = get_global_earthquakes()
print(df_global.head()) 


✅ 데이터가 'global_earthquakes.csv' 파일로 저장되었습니다!
            time  magnitude                                         place  \
0  1740837593979       5.00             22 km NE of Taytayan, Philippines   
1  1740835055757       4.40                                   Fiji region   
2  1740835025805       4.50                      21 km E of Shinjō, Japan   
3  1740833885750       4.27            144 km NNE of Vieques, Puerto Rico   
4  1740831485700       3.55  123 km NNE of Punta Cana, Dominican Republic   

       lat       lon    depth  
0   7.8935  126.6352   34.140  
1 -21.0123 -178.5754  527.973  
2  38.7729  140.5520   10.000  
3  19.6466  -65.3375   25.000  
4  19.6238  -67.9941   37.000  


In [37]:
from sklearn.preprocessing import StandardScaler

df_global = df_global.dropna()  
X = df_global[['lat', 'lon', 'depth']]  
y = (df_global['magnitude'] > 4.0).astype(int)  



In [38]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [39]:
from sklearn.neighbors import KNeighborsClassifier

# KNN 모델 학습
knn = KNeighborsClassifier(n_neighbors=20,weights='distance')
knn.fit(X_scaled, y)

KNeighborsClassifier(n_neighbors=20, weights='distance')

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# KNN 모델 학습
knn.fit(X_train, y_train)


y_pred = knn.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f" 모델 정확도: {accuracy:.4f}")


print("\n 분류 보고서 (Classification Report):")
print(classification_report(y_test, y_pred))

print("\n혼동 행렬 (Confusion Matrix):")
print(confusion_matrix(y_test, y_pred))


✅ 모델 정확도: 0.9218

📊 분류 보고서 (Classification Report):
              precision    recall  f1-score   support

           0       0.76      0.46      0.58        28
           1       0.93      0.98      0.96       215

    accuracy                           0.92       243
   macro avg       0.85      0.72      0.77       243
weighted avg       0.91      0.92      0.91       243


🧮 혼동 행렬 (Confusion Matrix):
[[ 13  15]
 [  4 211]]


In [29]:
import numpy as np

def predict_earthquake(lat, lon, depth=10.0):
    """
    새로운 좌표의 지진 발생 확률을 KNN이 자동으로 예측
    """
    input_df = pd.DataFrame([[lat, lon, depth]], columns=['lat', 'lon', 'depth'])
    input_scaled = scaler.transform(input_df)
    prob = knn.predict_proba(input_scaled)[0]  # 지진 발생 확률 (0~1)
    return round(prob[1] * 100, 2) if len(prob) > 1 else round(prob[0] * 100, 2)


In [ ]:
lat_test, lon_test = 37.5, 127.0  # 서울
test_prob = predict_earthquake(lat_test, lon_test)
print(f"📍 {lat_test}, {lon_test}의 지진 발생 확률: {test_prob}%")

📍 37.5, 127.0의 지진 발생 확률: 95.58%


In [ ]:
print(" 훈련 데이터 개수:", df_global.shape[0])


✅ 훈련 데이터 개수: 1211


In [ ]:
print(" 지진 발생 여부 비율:")
print(df_global['magnitude'].apply(lambda x: 1 if x > 4.5 else 0).value_counts(normalize=True))


✅ 지진 발생 여부 비율:
magnitude
0    0.656482
1    0.343518
Name: proportion, dtype: float64


In [ ]:
from sklearn.utils import resample

#  데이터를 분리
df_small_quakes = df_global[df_global['magnitude'] < 4.5]  
df_large_quakes = df_global[df_global['magnitude'] >= 4.5]  

In [ ]:

# Oversampling
if len(df_small_quakes) > 0:
    df_large_quakes_resampled = resample(df_large_quakes,
                                         replace=True,  # 복제 허용
                                         n_samples=len(df_small_quakes),  # 작은 지진과 개수 맞추기
                                         random_state=42)
else:
    df_large_quakes_resampled = df_large_quakes.copy()

In [ ]:

df_balanced = pd.concat([df_small_quakes, df_large_quakes_resampled])


count    1354.000000
mean        4.489697
std         0.425741
min         3.500000
25%         4.200000
50%         4.480000
75%         4.700000
max         7.600000
Name: magnitude, dtype: float64


In [ ]:
X = df_balanced[['lat', 'lon', 'depth']]
y = (df_balanced['magnitude'] >= 4.5).astype(int) 


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:

# KNN 학습 
knn = KNeighborsClassifier(n_neighbors=500) 
knn.fit(X_scaled, y)

KNeighborsClassifier(n_neighbors=500)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

knn.fit(X_train, y_train)


y_pred = knn.predict(X_test)



✅ 모델 정확도: 0.5609

📊 분류 보고서 (Classification Report):
              precision    recall  f1-score   support

           0       0.55      0.71      0.62       136
           1       0.58      0.41      0.48       135

    accuracy                           0.56       271
   macro avg       0.57      0.56      0.55       271
weighted avg       0.57      0.56      0.55       271


🧮 혼동 행렬 (Confusion Matrix):
[[96 40]
 [79 56]]


In [ ]:

accuracy = accuracy_score(y_test, y_pred)
print(f" 모델 정확도: {accuracy:.4f}")


print("\n 분류 보고서 (Classification Report):")
print(classification_report(y_test, y_pred))


print("\n혼동 행렬 (Confusion Matrix):")
print(confusion_matrix(y_test, y_pred))

In [ ]:
test_prob = predict_earthquake(37.5, 127.0)
print(f" 37.5, 127.0의 지진 발생 확률 (개선 후): {test_prob}%")


📍 37.5, 127.0의 지진 발생 확률 (개선 후): 43.6%


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest 모델 학습
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_scaled, y)

def predict_earthquake_rf(lat, lon, depth=10.0):
    input_df = pd.DataFrame([[lat, lon, depth]], columns=['lat', 'lon', 'depth'])
    input_scaled = scaler.transform(input_df)
    prob = rf.predict_proba(input_scaled)[0][1] 
    return round(prob * 100, 2)

# 서울
test_prob_rf = predict_earthquake_rf(37.5, 127.0)
print(f" 37.5, 127.0의 지진 발생 확률 (Random Forest): {test_prob_rf}%")


📍 37.5, 127.0의 지진 발생 확률 (Random Forest): 32.0%


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)


rf.fit(X_train, y_train)


y_pred_rf = rf.predict(X_test)


✅ Random Forest 모델 정확도: 0.8044

📊 분류 보고서 (Classification Report):
              precision    recall  f1-score   support

           0       0.83      0.77      0.80       136
           1       0.78      0.84      0.81       135

    accuracy                           0.80       271
   macro avg       0.81      0.80      0.80       271
weighted avg       0.81      0.80      0.80       271


🧮 혼동 행렬 (Confusion Matrix):
[[105  31]
 [ 22 113]]


In [ ]:
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f" Random Forest 모델 정확도: {accuracy_rf:.4f}")

print("\n 분류 보고서 (Classification Report):")
print(classification_report(y_test, y_pred_rf))


print("\n혼동 행렬 (Confusion Matrix):")
print(confusion_matrix(y_test, y_pred_rf))

In [ ]:
import joblib

#  1. 모델 저장 (Random Forest)
joblib.dump(rf, "earthquake_model.joblib")

#  2. 스케일러 저장 (데이터 정규화를 위해 필요)
joblib.dump(scaler, "scaler.joblib")

print(" 모델과 스케일러 저장 완료 (joblib 사용)")


✅ 모델과 스케일러 저장 완료 (joblib 사용)
